In [45]:
# Alternate image stitching Attempt - Zack H
# Accounts for camera shake/wobble, extremely intensive

In [46]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [47]:
# Functions
class Image_Stitching():
    def __init__(self) :
        self.ratio=0.85
        self.min_match=10
        self.sift=cv2.xfeatures2d.SIFT_create()
        self.smoothing_window_size=800

    def registration(self,img1,img2):
        kp1, des1 = self.sift.detectAndCompute(img1, None)
        kp2, des2 = self.sift.detectAndCompute(img2, None)
        matcher = cv2.BFMatcher()
        raw_matches = matcher.knnMatch(des1, des2, k=2)
        good_points = []
        good_matches=[]
        for m1, m2 in raw_matches:
            if m1.distance < self.ratio * m2.distance:
                good_points.append((m1.trainIdx, m1.queryIdx))
                good_matches.append([m1])
        img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, None, flags=2)
        cv2.imwrite('matching.jpg', img3)
        if len(good_points) > self.min_match:
            image1_kp = np.float32(
                [kp1[i].pt for (_, i) in good_points])
            image2_kp = np.float32(
                [kp2[i].pt for (i, _) in good_points])
            H, status = cv2.findHomography(image2_kp, image1_kp, cv2.RANSAC,5.0)
        return H
    
    def comb_img(self,img1, img2):
        x = 0
        y = 0
        newimg = img1
        while x < img1.shape[0]:
            while y < img1.shape[1]:
                #print(str(img1[x,y])," v ",str(img1[x,y]))
                if str(img1[x,y]) == '[0 0 0]':
                    #print(str(img2[x,y]))
                    if str(img2[x,y]) != '[0 0 0]':
                        print(str(img2[x,y]))
                        newimg[x,y] = img2[x,y]
                y += 1
            x += 1
        return newimg

    def mask_img(self,img1,img2):
        mat1 = np.zeros((img1.shape[0], img1.shape[1], 1))
        mat2 = np.zeros((img2.shape[0], img2.shape[1], 1))
        
        for i in range(0, img1.shape[0]-1):
            for j in range(0, img1.shape[1]-1):
                if max(img1[i,j]) > 0:
                    mat1[i,j] = 1
        for i in range(0, img2.shape[0]-1):
            for j in range(0, img2.shape[1]-1):
                if (max(img2[i,j]) > 0): 
                    mat2[i,j] = 1   
        mask = np.logical_and(mat1, mat2)        
        return mask
    
    def blending(self,img1,img2):
#         H = self.registration(img1,img2)
#         print("\nH:\n",H)
        
        height_panorama = img2.shape[0]*5
        width_panorama = img2.shape[1]*5
        ch = int(height_panorama/2) #center height + width
        cw = int(width_panorama/2)

        panorama1 = np.zeros((height_panorama, width_panorama, 3))
        panorama2 = np.zeros((height_panorama, width_panorama, 3))
        
        off2h = 0
        off2w = 0
        off1h = 0
        off1w = 0
        
        #fix rounding error when centering image
        if img2.shape[0] % 2 != 0:
            off2h = 1
        if img2.shape[1] % 2 != 0:
            off2w = 1
        if img1.shape[0] % 2 != 0:
            off1h = 1
        if img1.shape[1] % 2 != 0:
            off1w = 1        
        
        panorama1[ch-int(img2.shape[0]/2):ch+int(img2.shape[0]/2)+off2h, cw-int(img2.shape[1]/2):cw+int(img2.shape[1]/2)+off2w, :] = img2
        panorama2[ch-int(img1.shape[0]/2):ch+int(img1.shape[0]/2)+off1h, cw-int(img1.shape[1]/2):cw+int(img1.shape[1]/2)+off1w, :] = img1 #center images in panorama
        
        panorama1 = (panorama1).astype(np.uint8)
        panorama2 = (panorama2).astype(np.uint8)
        
        H = self.registration(panorama1,panorama2)
        
        panorama2 = cv2.warpPerspective(panorama2, H, (width_panorama, height_panorama))
        
        rows, cols = np.where(img1[:, :, 0] != 0)
        min_row, max_row = min(rows), max(rows) + 1
        min_col, max_col = min(cols), max(cols) + 1        
        dim1 = [max_row-min_row, max_col-min_col]
        
        rows, cols = np.where(panorama2[:, :, 0] != 0)
        min_row, max_row = min(rows), max(rows) + 1
        min_col, max_col = min(cols), max(cols) + 1        
        dim2 = [max_row-min_row, max_col-min_col]
        
        print("Width:",dim1[1],"v",dim2[1],", Height:",dim1[0]," v ",dim2[0])
        
        if (dim2[0] > 3*dim1[0]) or (dim2[1] > 3*dim1[1]) or (dim2[0] < 0.4*dim1[0]) or (dim2[1] < 0.4*dim1[1]):
            print("> WARNING: New image too distorted, discarding.") #only allow up to 10% distortion of image or discard
            return(img2)
        
        #res = cv2.bitwise_and(img,img,mask = mask)
        
        #img1 /= img1.max()/255.0
        panorama1 = (panorama1).astype(np.uint8)
        panorama2 = (panorama2).astype(np.uint8)
        
#         print("\n-------------------------\nPan1:\n")
#         plt.imshow(panorama1); 
#         plt.show() 
#         print(panorama1.max())
#         print("Pan2:\n")
#         plt.imshow(panorama2); 
#         plt.show() 
#         print(panorama2.max())
        
        mask = self.mask_img(panorama1, panorama2)
        panorama2 = panorama2*np.invert(mask)
        #panorama1 = panorama1*mask
        
        #result = self.comb_img(panorama1,panorama2)
        result = panorama1+panorama2
        
#         print("res:\n")
#         plt.imshow(result); 
#         plt.show() 
#         print(result.max())
        
        rows, cols = np.where(result[:, :, 0] != 0)
        min_row, max_row = min(rows), max(rows) + 1
        min_col, max_col = min(cols), max(cols) + 1
        final_result = result[min_row:max_row, min_col:max_col, :]
        return final_result

In [48]:
# Vars
#wdir = "F:\\frames\\transformed\\conv"
wdir = "I:\\Summer 2019\\Capstone\\Frames-DemoTest\\2b\\calibrated"
odir = wdir + "\\stitched\\"
ext = ".jpg"

In [49]:
# Main, loops over bitmaps and stitches

img1 = None
img2 = None

if odir and not os.path.isdir(odir):
    os.mkdir(odir)

directory = os.fsencode(wdir)

print("Processing...")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print(filename)
    if filename.endswith(ext):        
        img2 = img1
        img1 = cv2.imread(wdir + "\\"+ filename)
        #print("img1\n",img1)
        #print("img2\n",img2)
        if img1 is None:
            print("> WARNING, image empty.")
            continue
        if img2 is None:
            continue
        img1 = Image_Stitching().blending(img1,img2)
        #img1 /= img1.max()/255.0
        #img1 = (img1).astype(np.uint8)
        cv2.imwrite(odir + "stitched-"+filename, img1)
        #plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)); #show color flipped image
        #plt.show()  
    
print("Done.")

Processing...
output_0000274.jpg
output_0000275.jpg
Width: 1792 v 1757 , Height: 862  v  837
output_0000276.jpg
Width: 1792 v 1544 , Height: 862  v  944
output_0000277.jpg
Width: 1792 v 1501 , Height: 862  v  610
output_0000278.jpg
Width: 1792 v 9120 , Height: 862  v  3829
> WARNING: New image too distorted, discarding.
output_0000279.jpg
Width: 1792 v 3169 , Height: 862  v  1154
output_0000280.jpg
Width: 1792 v 15845 , Height: 862  v  6660
> WARNING: New image too distorted, discarding.
output_0000281.jpg
Width: 1792 v 15845 , Height: 862  v  6660
> WARNING: New image too distorted, discarding.
output_0000282.jpg
Width: 1792 v 15845 , Height: 862  v  6660
> WARNING: New image too distorted, discarding.
output_0000283.jpg
Width: 1792 v 15845 , Height: 862  v  6660
> WARNING: New image too distorted, discarding.
output_0000284.jpg
Width: 1792 v 15845 , Height: 862  v  6660
> WARNING: New image too distorted, discarding.
output_0000285.jpg
Width: 1792 v 12746 , Height: 862  v  6660
> WAR

KeyboardInterrupt: 

In [ ]:
# str(img1[1,2])

In [ ]:
plt.imshow(img1); 
plt.show() 
plt.imshow(img2); 
plt.show() 
img1.max()

In [ ]:
# print("img1\n",img1)
# print("img2\n",img2)

In [ ]:
#COLOR TEST
img1 = [[0]]
img1[0][0] = [0,0,255] 
plt.imshow(img1); 
plt.show() 

In [ ]:
img1

In [ ]:
rows, cols = np.where(img2[:, :, 0] != 0)
        min_row, max_row = min(rows), max(rows) + 1
        min_col, max_col = min(cols), max(cols) + 1   
        leftcorner = np.where(img2[:,mincol,0] != 0)

plt.imshow(img2); 
plt.show() 

plt.imshow(crop); 
plt.show() 